## 1. Importar las Librerías Necesarias

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
from wordcloud import WordCloud
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import log_loss, accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from xgboost import XGBClassifier
from sklearn.metrics import cohen_kappa_score
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
import 


%matplotlib inline
pd.set_option('display.max_columns', 30)
plt.rcParams['figure.figsize'] = [12.0, 8.0]
print ("librerias cargadas")

Leer Jason

In [ ]:
!pip install ijson
import numpy as np
import pandas as pd
import ijson
import json
from pandas.io.json import json_normalize
print ("librerias Json cargadas")

In [ ]:
%%bash
# we can use %%bash magic to print a preview of our file

head ../input/petfinder-adoption-prediction/train_sentiment/0008c5398.json

In [ ]:
#import glob

#train_sentiment_files = sorted(glob.glob('../input/petfinder-adoption-prediction/train_sentiment/*.json'))
#print('num of train sentiment files: {}'.format(len(train_sentiment_files)))
#data=[]


**MIRAR ESTE CODIGO PARA SENTIMENT ANALYSIS TRAINING
 # Hay que importar glob, se puede poner arriba o dejar como está**

In [ ]:
import glob

train_sentiment_files = sorted(glob.glob('../input/petfinder-adoption-prediction/train_sentiment/*.json'))
print('num of train sentiment files: {}'.format(len(train_sentiment_files)))
data=[]

for i in range (len(train_sentiment_files)):
    with open(train_sentiment_files[i]) as jsonFile:
        jsonObject = json.load(jsonFile)
        jsonFile.close()
    score=jsonObject['documentSentiment']['score']
    magnitude=jsonObject['documentSentiment']['magnitude']
    path=train_sentiment_files[i]
    firstpos=path.rfind("/")
    lastpos=path.rfind(".")
    PetID=path[firstpos+1:lastpos]
    data.append([PetID,score,magnitude,score*magnitude])

data = pd.DataFrame(data, columns=['PetID','Score', 'Magnitude','Sentiment'])
data.head(5)

#train_df=pd.merge(train_df, data, on='PetID',how='outer')
#train_df=train_df.fillna(0)

In [ ]:
data.head(5)

## 2. Leer los Datos
Al menos los datos Tabulares de la base de "train"

In [ ]:
#por el momento solo leemos los datos tabulares 
train = pd.read_csv('../input/petfinder-adoption-prediction/train/train.csv')

## 3. Pre-procesar Nulos
Verificar la existencia de Nulos y decidir como Imputarlos en caso de que existan

Verificar la existencia de Ceros u otros valores que puedan indicar que pueden ser perdidos

In [ ]:
#vemos primero que valiables tienen nulos
train.isnull().sum()

#al parecer solo nombre y descripcion tienen nulos. Es por eso que vamos a considerar no tener nombre y no tener descripcion 
#como una variable mas para usar de input del modelo. 

In [ ]:
train['Tiene_nombre?'] = np.where(train['Name'].notnull(), 0, 1)
train['Tiene_descripcion?'] = np.where(train['Description'].notnull(), 0, 1)

In [ ]:
train.info()

In [ ]:
train.describe()

###outliers###

#Age
#no vamos a hacer ningun tratamiento de esto, dado que para lightgbm no es necesario

###varialbes a Codificar###

#maturitysyze
#Vaccinated
#Dewormed
#sterilized
#health

####variable nueva###

#vacunated & dewormed
#vacunated & sterilezed
#sterilezed & dewormed
#sterlezed & dewormed & vacunated
#has photo?
#Has video?
#Has photo & video_?


In [ ]:
###varialbes a Codificar###
OneHotEncoder().fit_transform(train[["MaturitySize", "Vaccinated","Dewormed","Sterilized", "Health"]])

#codificacion de variables

encoder = OneHotEncoder().fit(train[["MaturitySize", "Vaccinated","Dewormed","Sterilized", "Health"]])


In [ ]:
codificadas = pd.DataFrame(encoder.transform(train[["MaturitySize", "Vaccinated","Dewormed","Sterilized", "Health"]]).toarray(),
            columns=[
                f"{column}_{category}" for categories, column in zip(encoder.categories_, train[["MaturitySize", "Vaccinated","Dewormed","Sterilized", "Health"]].columns)
                for category in categories
            ])

In [ ]:
train.head()

In [ ]:
train = pd.concat([train, codificadas], axis=1)

In [ ]:
train.info()

In [ ]:
####variable nueva###
train['Vaccinated&Dewormed'] = np.where((train['Vaccinated'] == 1) & (train['Dewormed']==1), 1, 0)
train['Vaccinated&Sterilized'] = np.where((train['Vaccinated'] == 1) & (train['Sterilized']==1), 1, 0)
train['Sterilized&Dewormed'] = np.where((train['Sterilized'] == 1) & (train['Dewormed']==1), 1, 0)
train['Sterilized&Dewormed&Vaccinated'] = np.where((train['Vaccinated'] == 1) & (train['Dewormed']==1)& (train['Sterilized']==1), 1, 0)
train['Hasphoto?'] = np.where((train['PhotoAmt'] > 0), 1, 0)
train['Hasvideo?'] = np.where((train['VideoAmt'] > 0), 1, 0)
train['Hasvideo&photo?'] = np.where((train['VideoAmt'] > 0) & (train['PhotoAmt'] > 0), 1, 0)

## 4. Convertir o eliminar las Columnas Categóricas

Por ejemplo, la Descripción habría que sacarla para un análisis independiente

In [ ]:
#variables a eliminar
eliminar = ["Name","RescuerID","Description","PetID"]
data = train.drop(eliminar, axis=1)
print(data.columns.values)

## 5. Normalizar o Estandarizar las variables Numericas (para los modelos que sean necesarios)

Revisar si existen valores extremos y considerarlos para los modelos que afecte

In [ ]:
#para este modelo no vamos a necesitar esto.

## 6. Separa la base de Test (10%) y Train (90%)
Pueden ser otros porcentajes que les parezcan mejor

In [ ]:
X= data.loc[:, data.columns != 'AdoptionSpeed']
y= data['AdoptionSpeed']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=2021)

### 7. Para la parte de Train, armar un esquema de Cross Validation

Usar 10 Folds

In [ ]:
def metric(y_true, y_pred):
    res = cohen_kappa_score(y_true, y_pred.reshape((y_true.shape[0], 5), order="F").argmax(axis=1), weights= 'quadratic')
    return "kappa", res, True

In [ ]:
pipe = Pipeline([("classifier",LGBMClassifier(n_estimators=200, metric="custom",verbose=50, early_stopping_rounds=20) )])


In [ ]:
search_space = [
    {
        "classifier": [LGBMClassifier(n_estimators=50, metric="custom")],
        "classifier__max_depth": list(range(4, 9, 1)),
      #  "classifier__min_data_in_leaf": list(range(38,50,2)),
       # "classifier__learning_rate": [0.1,0.15,0.2,0.25]
    }
]
#primer intento
#LGBMClassifier(learning_rate=0.015, max_depth=7, metric='custom',
#               min_data_in_leaf=40, n_estimators=300)
#segundo intento
#LGBMClassifier(learning_rate=0.017, max_depth=6, metric='custom',
#               min_data_in_leaf=45, n_estimators=300)
#tercer intento 
#LGBMClassifier(learning_rate=0.022, max_depth=6, metric='custom',
#               min_data_in_leaf=44, n_estimators=300)
#cuarto intento
#LGBMClassifier(learning_rate=0.05, max_depth=5, metric='custom',
#               min_data_in_leaf=46, n_estimators=200)

In [ ]:
clf = GridSearchCV(pipe, search_space, cv=10, verbose=30)

## 8. Entrenar al menos un Modelo que prefieran y optimizar al menos un Hiperparámetro

In [ ]:
mejor_modelo = clf.fit(X_train, y_train)
print(mejor_modelo.best_estimator_.get_params()["classifier"])

Probamos con el set de testing

In [ ]:
#por el momento solo leemos los datos tabulares 
test = pd.read_csv('../input/petfinder-adoption-prediction/test/test.csv')

In [ ]:
test['Tiene_nombre?'] = np.where(test['Name'].notnull(), 0, 1)
test['Tiene_descripcion?'] = np.where(test['Description'].notnull(), 0, 1)
###varialbes a Codificar###
OneHotEncoder().fit_transform(test[["MaturitySize", "Vaccinated","Dewormed","Sterilized", "Health"]])

#codificacion de variables

encoder = OneHotEncoder().fit(test[["MaturitySize", "Vaccinated","Dewormed","Sterilized", "Health"]])

codificadas = pd.DataFrame(encoder.transform(test[["MaturitySize", "Vaccinated","Dewormed","Sterilized", "Health"]]).toarray(),
            columns=[
                f"{column}_{category}" for categories, column in zip(encoder.categories_, test[["MaturitySize", "Vaccinated","Dewormed","Sterilized", "Health"]].columns)
                for category in categories
            ])

test = pd.concat([test, codificadas], axis=1)

####variable nueva###
test['Vaccinated&Dewormed'] = np.where((test['Vaccinated'] == 1) & (test['Dewormed']==1), 1, 0)
test['Vaccinated&Sterilized'] = np.where((test['Vaccinated'] == 1) & (test['Sterilized']==1), 1, 0)
test['Sterilized&Dewormed'] = np.where((test['Sterilized'] == 1) & (test['Dewormed']==1), 1, 0)
test['Sterilized&Dewormed&Vaccinated'] = np.where((test['Vaccinated'] == 1) & (test['Dewormed']==1)& (test['Sterilized']==1), 1, 0)
test['Hasphoto?'] = np.where((test['PhotoAmt'] > 0), 1, 0)
test['Hasvideo?'] = np.where((test['VideoAmt'] > 0), 1, 0)
test['Hasvideo&photo?'] = np.where((test['VideoAmt'] > 0) & (test['PhotoAmt'] > 0), 1, 0)



In [ ]:
#variables a eliminar
sacar = ["Name","RescuerID","Description","PetID"]
data2 = test.drop(sacar, axis=1)
print(data2.columns.values)

In [ ]:
predictions = mejor_modelo.predict(data2)
#predictions = optR.predict(predictions)

In [ ]:
submission = test[['PetID']].copy()
submission["AdoptionSpeed"] = predictions.astype("int64")
submission.to_csv("submission.csv", index=False)

submission["AdoptionSpeed"].value_counts(normalize=True,dropna=False)